In [2]:
import sys
import os
import numpy as np
import pandas as pd
import string
from collections import Counter 
import json
import pickle
import zipfile

In [3]:
# Compute the note placements in one beat map and return a list of placements
def compute_note_placements(dat_json):
    # List of all notes, not grouped with notes at same times
    notes_list = dat_json['_notes']
    # List of all unique time points that notes are at
    note_timings = set(notes_list['_time'])
    # Dictonary mapping time point to list of notes at that time. Beat saber has 3x4 grid of note positions (=12)
    notes_at_time_point = {note_timing : [0] * 12 for note_timing in note_timings}
    for note in notes_list:
        # If it is a bomb then skip as our model doesn't deal with bombs
        if note['_type'] == 3:
            continue
        # 0 - Red, 1 - Blue
        colour = note['_type'] 
        # 0 - Down, 1 - Up, 2 - Right, 3 - Left,
        # 4 - Bottom-Right, 5 - Bottom-Left, 6 - Top-Right, 7 - Top-Left
        # 8 - No Direction
        direction = note['_cutDirection']
        # Integer classification based on colour and direction. (Colour * 9 since 9 directions per colour)
        note_type = colour * 9 + direction + 1 # Plus 1 to account for 0 being no note
        
        # Ranges from 0 to 3 (3x4 grid)
        row = note['lineIndex']
        # Ranges from 0 to 2 (3x4 grid)
        col = note['lineLayer']

        # Update the dictionary with the location and type of note
        notes_at_time_point[note['_time']][row * 4 + col] = note_type
    
    return list([tuple(time_point.values() for time_point in notes_at_time_point)])


In [4]:
# Compute the most common note placement across many beat maps
def compute_most_common_note_placements(maps_df, max_maps=70000):
    maps_processed = 0
    # Counter of tuples representing the placement and types of blocks
    total_placements = Counter()
    for file_path in maps_df.file_path.unique():
        if maps_processed > max_maps:
            break
        if file_path != "NOT_FOUND":
            with ZipFile(file_path) as folder:
                filenames = folder.namelist()
                difficulties_dats = [diff for diff in filenames if diff in ["Expert.dat", "ExpertPlus.dat"]]
                for difficulty_dat in difficulties_dats:
                    with folder.open(difficulty_dat) as diff_dat:
                        dat_json = json.load(diff_dat)
                        placement_list = compute_note_placements(dat_json)
                        total_placements.update(placement_list)
                        maps_processed += 1



In [5]:
# Get maps dataframe from the pickle file
maps_df = pd.read_pickle("downloaded_maps_df.pkl")